In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader


from google.colab import drive
drive.mount('/content/drive')


main_dataset = '/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset'

file_paths = []
labels = []

if os.path.exists(main_dataset):
    print('Path exists, conducting extraction of image data for processing. I decide to go the path of Gaussian NB using Historgrams')

    for folder in os.listdir(main_dataset):
        class_path = os.path.join(main_dataset, folder)

        if os.path.isdir(class_path):
            print(f'{class_path} added to processing queue')

            for image_file in os.listdir(class_path):
                if image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    image_path = os.path.join(class_path, image_file)
                    file_paths.append(image_path)
                    labels.append(folder)
else:
    print('Path does not exist - you need to try again')

unique_classes = list(set(labels))
print(f'Total classes: {len(unique_classes)}')
print(f'Total images: {len(file_paths)}')
print(f'Classes found: {unique_classes}')

for class_name in unique_classes:
    count = labels.count(class_name)
    print(f'{class_name}: {count} images')

Mounted at /content/drive
Path exists, conducting extraction of image data for processing. I decide to go the path of Gaussian NB using Historgrams
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset/banana added to processing queue
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset/balloon added to processing queue
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset/backpack added to processing queue
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset/blocks added to processing queue
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset/apple added to processing queue
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset/bag added to processing queue
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Vision Project/dataset/blanket added to processing queue
/content/drive/MyDrive/Colab Notebooks/AIPI 540/Computer Visi

In [ ]:
def extract_color_histogram(image_path, bins=32):
    image = Image.open(image_path)
    image = image.resize((128, 128))
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image_array = np.array(image)

    hist_r = np.histogram(image_array[:,:,0], bins=bins, range=(0, 256))[0]
    hist_g = np.histogram(image_array[:,:,1], bins=bins, range=(0, 256))[0]
    hist_b = np.histogram(image_array[:,:,2], bins=bins, range=(0, 256))[0]

    hist_r = hist_r / np.sum(hist_r)
    hist_g = hist_g / np.sum(hist_g)
    hist_b = hist_b / np.sum(hist_b)

    color_features = np.concatenate([hist_r, hist_g, hist_b])
    return color_features

features = []
processed_labels = []

for i, image_path in enumerate(file_paths):
    try:
        color_hist = extract_color_histogram(image_path, bins=32)
        features.append(color_hist)
        processed_labels.append(labels[i])
        if i % 50 == 0:
            print(f"Processed {i+1}/{len(file_paths)} images")
    except Exception as e:
        print(f"Error processing.') {image_path}: {e}")
        continue

X = np.array(features)
y = np.array(processed_labels)

print(f"Feature extraction complete!")
print(f"Feature matrix shape: {X.shape}")
print(f"Total samples: {len(y)}")


print(" Training and evaluating traditional ML models")

le = LabelEncoder()
y_encoded = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print("GAUSSIAN NB:")

gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_pred = gnb.predict(X_test)
gnb_accuracy = accuracy_score(y_test, gnb_pred)

print(f"Accuracy: {gnb_accuracy:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, gnb_pred))
print("Classification Report:")
unique_test_labels = sorted(list(set(y_test)))
test_class_names = [le.classes_[i] for i in unique_test_labels]
print(classification_report(y_test, gnb_pred, labels=unique_test_labels, target_names=test_class_names, zero_division=0))

print("SVC(I had wanted to perform GridSearch but processessing took too long)")

svc = SVC(kernel='rbf', random_state=42)
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)
svc_accuracy = accuracy_score(y_test, svc_pred)

print(f"Accuracy: {svc_accuracy:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, svc_pred))
print("Classification Report:")
print(classification_report(y_test, svc_pred, labels=unique_test_labels, target_names=test_class_names, zero_division=0))

Processed 1/6976 images
Processed 51/6976 images
Processed 101/6976 images
Processed 151/6976 images
Processed 201/6976 images
Processed 251/6976 images
Processed 301/6976 images
Processed 351/6976 images
Processed 401/6976 images
Processed 451/6976 images
Processed 501/6976 images
Processed 551/6976 images
Processed 601/6976 images
Processed 651/6976 images
Processed 701/6976 images
Processed 751/6976 images
Processed 801/6976 images
Processed 851/6976 images
Processed 901/6976 images
Processed 951/6976 images
Processed 1001/6976 images
Processed 1051/6976 images
Processed 1101/6976 images
Processed 1151/6976 images
Processed 1201/6976 images
Processed 1251/6976 images
Processed 1301/6976 images
Processed 1351/6976 images
Processed 1401/6976 images
Processed 1451/6976 images
Processed 1501/6976 images
Processed 1551/6976 images
Processed 1601/6976 images
Processed 1651/6976 images
Processed 1701/6976 images
Processed 1751/6976 images
Processed 1801/6976 images
Processed 1851/6976 imag

In [ ]:


class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, y_encoded, test_size=0.2, random_state=42)

train_dataset = ImageDataset(train_paths, train_labels, transform)
test_dataset = ImageDataset(test_paths, test_labels, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(le.classes_))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Training ResNet50...")
model.train()
for epoch in range(100):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/100, Loss: {running_loss/len(train_loader):.4f}")

model.eval()
resnet_pred = []
resnet_true = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        resnet_pred.extend(predicted.cpu().numpy())
        resnet_true.extend(labels.cpu().numpy())

resnet_accuracy = accuracy_score(resnet_true, resnet_pred)

print(f"Accuracy: {resnet_accuracy:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(resnet_true, resnet_pred))
print("Classification Report:")
resnet_unique_labels = sorted(list(set(resnet_true)))
resnet_class_names = [le.classes_[i] for i in resnet_unique_labels]
print(classification_report(resnet_true, resnet_pred, labels=resnet_unique_labels, target_names=resnet_class_names, zero_division=0))


print("Model Comparisn:")

print(f"Gaussian Naive Bayes: {gnb_accuracy:.4f}")
print(f"Support Vector Classifier: {svc_accuracy:.4f}")
print(f"ResNet50 (PyTorch): {resnet_accuracy:.4f}")


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 209MB/s]


Training ResNet50...


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/100, Loss: 4.5445
Epoch 2/100, Loss: 4.1203
Epoch 3/100, Loss: 3.7590
Epoch 4/100, Loss: 3.4368
Epoch 5/100, Loss: 3.0938
Epoch 6/100, Loss: 2.8130
Epoch 7/100, Loss: 2.5087
Epoch 8/100, Loss: 2.2530
Epoch 9/100, Loss: 1.9729
Epoch 10/100, Loss: 1.7319
Epoch 11/100, Loss: 1.4639
Epoch 12/100, Loss: 1.1830
Epoch 13/100, Loss: 0.9427
Epoch 14/100, Loss: 0.7382
Epoch 15/100, Loss: 0.5581
Epoch 16/100, Loss: 0.4194
Epoch 17/100, Loss: 0.3158
Epoch 18/100, Loss: 0.2543
Epoch 19/100, Loss: 0.2319
Epoch 20/100, Loss: 0.2367
Epoch 21/100, Loss: 0.2248
Epoch 22/100, Loss: 0.2087
Epoch 23/100, Loss: 0.1672
Epoch 24/100, Loss: 0.1788
Epoch 25/100, Loss: 0.1481
Epoch 26/100, Loss: 0.1251
Epoch 27/100, Loss: 0.1530
Epoch 28/100, Loss: 0.1366
Epoch 29/100, Loss: 0.1583
Epoch 30/100, Loss: 0.1668
Epoch 31/100, Loss: 0.1233
Epoch 32/100, Loss: 0.1266
Epoch 33/100, Loss: 0.1299
Epoch 34/100, Loss: 0.1716
Epoch 35/100, Loss: 0.1440
Epoch 36/100, Loss: 0.0739
Epoch 37/100, Loss: 0.0625
Epoch 38/1